In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

plt.style.use('default')

sns.set(style='whitegrid')

# Análisis del Conversion Funnel de usuarios que visitan el sitio y realizan una conversion en el mismo.

In [ ]:
df = pd.read_csv('data/events.csv', low_memory=False)

In [ ]:
# TODO: Cargar usuarios con conversion y checkout, mostrar que el top tiene conversions sin checkouts
# y plantear la razón posible para esto, a modo de introducción al análisis y filtro que haremos para el mismo.

In [ ]:
# Vamos a obtener a todos los usuarios que tienen conversiones, para luego obtener a aquellos que también
# tienen check-outs.
conversiones = df.loc[df['event'] == 'conversion', ['event', 'person', 'sku', 'model']].copy()
#eventos = eventos[['event', 'person']]
conversiones_sin_us_dup = conversiones.drop_duplicates(subset='person')
usuarios_con_conversiones = set(conversiones_sin_us_dup['person'])

In [ ]:
# Ahora obtenemos los que también tienen check-outs.
usuarios_con_checkout_conversion = df.loc[(df['event'] == 'checkout') &
                                          df['person'].isin(usuarios_con_conversiones), ['person']]
usuarios_con_checkout_conversion.drop_duplicates(subset='person', inplace=True)
usuarios_con_checkout_conversion = set(usuarios_con_checkout_conversion['person'])

In [ ]:
# Procedemos a quedarnos con aquellos que compraron en el sitio de la empresa, nos basamos en que
# exista un check-out previo a una conversion, sobre el mismo SKU.
usuarios_filtrados = df.loc[(df['person'].isin(usuarios_con_checkout_conversion)) &
            (df['event'].isin(['checkout', 'conversion'])), ['timestamp', 'event', 'person', 'sku', 'model']]
usuarios_filtrados.drop_duplicates(subset=['event','person'], inplace=True)
usuarios_filtrados = usuarios_filtrados[usuarios_filtrados.duplicated(subset=['person', 'sku'], keep=False)]
usuarios_filtrados.describe().T

In [ ]:
# Notemos que algunos usuarios tienen invertido el orden check-out/conversion. Por lo que no sabemos si
# la conversion fue realizada en el sitio o fuera del mismo (y presentan una diferencia de tiempo importante
# entre ambos eventos)
usuarios_filtrados.head()

In [ ]:
# Obtenemos una lista de usuarios sin duplicados.
usuarios = set(usuarios_filtrados.drop_duplicates(subset='person')['person'])
len(usuarios)

In [ ]:
# Ahora nos quedamos con aquellos que tienen conversions pero no tienen check-outs relacionados a estas.
us_conversion_sin_checkout = set([x for x in usuarios_con_conversiones if x not in usuarios])
len(us_conversion_sin_checkout)

In [ ]:
# Nos quedaría quitar a aquellos usuarios que tienen invertido el orden en el que se realiza una conversion
# desde el sitio de la empresa. Es decir, aquellos que primero presenten una conversion y luego un check-out.
usuarios_conflictivos = set()
row_iterator = usuarios_filtrados.iterrows()
for i, row in row_iterator:
    next_row = next(row_iterator)[1]
    if (row['event'] + next_row['event']) == "conversioncheckout":
        usuarios_conflictivos.add(row['person'])
us_conversion_sin_checkout = us_conversion_sin_checkout.union(usuarios_conflictivos)
len(us_conversion_sin_checkout)